## LAB 9
### Podstawowy algorytm ewolucyjny

In [119]:
import numpy as np
from typing import Callable
import random

In [180]:
class Genetic:
    
    def __init__(self, 
                 cost_function: Callable,
                 problem_dimension: int,
                 n_population: int,
                 p_mutation: float,
                 p_crossover: float):
        
        self.cost_function = cost_function
        self.problem_dimension = problem_dimension
        self.n_population = n_population
        self.p_mutation = p_mutation
        self.p_crossover = p_crossover
        # Initialization of the population
        self.population = np.random.uniform(-10, 10, size=(n_population, problem_dimension))
        
    def fit(self, n_iter):
        """Execution of genetic algorithm.
        
        Stop condition: number of iterations.
        Crossover: one-point.
        Mutation: gaussian.
        Selection: binary tournaments, with parents, with replacement.
        """
        
        for i in range(n_iter):
            # Crossover
            size = np.floor(self.p_mutation*self.n_population)
            # Even number of individuals for crossover
            size = int(size - size%2)
            crossover_ids = np.random.choice(range(self.n_population), 
                                             size=size,
                                             replace=False)
            children = []
            for j in range(0, size-1, 2):
                # Switching ceil(half) parts of individuals
                point = int(np.ceil(self.problem_dimension/2))
                part_1 = self.population[crossover_ids[j]][:point]
                part_2 = self.population[crossover_ids[j+1]][point:]
                child = np.append(part_1, part_2)
                # Possible mutation
                if random.random()<self.p_mutation:
                    child = child + np.random.normal(size=(self.problem_dimension))
                children.append(child)
            
            # Combinig old and new generations
            children = np.array(children)
            both = np.append(self.population, children, axis=0)
            
            # Evaluation
            values = []
            for x in both:
                values.append(self.cost_function(x))
            
            # Selection
            new_population = []
            count = 0
            while count<self.n_population:
                ids = np.random.choice(range(len(both)), 
                                       size=2, 
                                       replace=False)
                if values[ids[0]]<=values[ids[1]]:
                    new_population.append(both[ids[0]])
                else:
                    new_population.append(both[ids[1]])
                count+=1
            
            self.population = np.array(new_population)
            
        return (both[np.argmin(values)], np.min(values))

In [112]:
def problem1_fun(ind):
    return ind[0]**2+ind[1]**2+2*ind[2]**2

In [185]:
gen = Genetic(problem1_fun, 3, 200, 0.2, 0.7)

In [186]:
gen.fit(50)

(array([-0.26072084, -0.00247476,  0.06603037]), 0.07670150270029419)